<a href="https://colab.research.google.com/github/dattali18/IR_Assignments/blob/main/Assignment.01/notebook/01_2_IR_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IR - Assignment 1

## Part 2 - DF IDF

In this part we will use the TF-IDF with BM25 and Okapi Algorithm

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# get the cleaned data from the source

aj_word_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word1/A_J_word.csv?raw=true"
bbc_word_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word1/BBC_word.csv?raw=true"
jp_word_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word1/J_P_word.csv?raw=true"
nyt_word_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word1/NYT_word.csv?raw=true"

In [ ]:

aj_lemma_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/lemma1/A_J_lemma.csv?raw=true"
bbc_lemma_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/lemma1/BBC_lemma.csv?raw=true"
jp_lemma_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/lemma1/J_P_lemma.csv?raw=true"
nyt_lemma_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/lemma1/NYT_lemma.csv?raw=true"

In [ ]:
aj_word_file = "A_J_word.csv"
bbc_word_file = "BBC_word.csv"
jp_word_file = "J_P_word.csv"
nyt_word_file = "NYT_word.csv"

In [ ]:
aj_lemma_file = "A_J_lemma.csv"
bbc_lemma_file = "BBC_lemma.csv"
jp_lemma_file = "J_P_lemma.csv"
nyt_lemma_file = "NYT_lemma.csv"

In [ ]:
# download the data using requests
import requests
import pandas as pd
import os

response = requests.get(aj_word_link)
if response.status_code == 200: # if the response is 200 OK
  with open(aj_word_file, "wb") as file:
    file.write(response.content)
  print(f"Sucessfully downloaded the file into {aj_word_file}")

Sucessfully downloaded the file into A_J_word.csv


In [ ]:
# open the file using pandas

df_aj = pd.read_csv(aj_word_file)
df_aj.head()

,id,document
0,aj_1,"pope renews call for gaza ceasefire , release ..."
1,aj_2,biden is still the best us president israel co...
2,aj_3,israeli air strikes continue across gaza as tr...
3,aj_4,police remove pro - palestinian students from ...
4,aj_5,mass graves found at southern gaza hospital ra...


We will build our own TF-IDF BM25 algorithm

$$
f(q, d)=\sum_{w\in q\cap d}{\frac{(k+1)c(w,d)}{c(w,d) +k\left(1 - b +b\cdot \frac{L_d}{L_{avg}}\right)}} \cdot \log\left(\frac{M + 1}{df(w)}\right)
$$

$$b\in[0, 1], k\in[0, +\infty)$$


In [ ]:
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download("punkt")
nltk.download("stopwords")
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
corpus = df_aj['document'].tolist()

In [ ]:
# process the corpus according the requirments
# remove the stop words from the documents

import re

stop_words = set(stopwords.words("english"))

def process_corpus(corpus):
  processed_corpus = []

  for doc in corpus:
    # Convert to lowercase
    doc = doc.lower()
    # Remove punctuation
    doc = re.sub(r'[^\w\s]', '', doc)
    # Remove numbers
    doc = re.sub(r'\d+', '', doc)
    # Tokenize the document
    words = word_tokenize(doc)
    # Remove stop words
    words = [word for word in words if word not in stopwords.words('english')]
    # Join the words back into a string
    processed_doc = ' '.join(words)
    processed_corpus.append(processed_doc)
  return processed_corpus

In [ ]:
# process the corpus

processed_corpus = process_corpus(corpus)

In [ ]:
vocab = set()
for doc in processed_corpus:
    words = doc.split()
    words = [w.lower() for w in words ]
    vocab.update(words)

# order the set and convert to list
vocab = list(vocab)
vocab.sort()

In [ ]:
def get_vocab(corpus):
  vocab = set()
  for doc in corpus:
    words = doc.split()
    words = [w.lower() for w in words ]
    vocab.update(words)

  vocab = list(vocab)
  vocab.sort()
  return vocab

In [ ]:
print(len(vocab))

4393


In [ ]:
print(vocab)

['aa', 'abadi', 'abbas', 'abc', 'abdallah', 'abdel', 'abductors', 'abide', 'ability', 'able', 'abound', 'abroad', 'absent', 'absolute', 'abstain', 'abstains', 'abstention', 'absurd', 'abu', 'abuqamar', 'abuse', 'abused', 'abuses', 'abusive', 'academic', 'academics', 'academy', 'accelerated', 'accept', 'accepted', 'accepting', 'accepts', 'access', 'accordance', 'according', 'accountability', 'accounts', 'accusations', 'accuse', 'accused', 'accuses', 'accusing', 'achieve', 'achieved', 'achieving', 'acknowledge', 'acre', 'acres', 'across', 'act', 'action', 'actions', 'activated', 'activist', 'activists', 'activity', 'acts', 'actually', 'acute', 'add', 'adding', 'addition', 'additional', 'address', 'addressed', 'addresses', 'adds', 'aden', 'adequate', 'adha', 'administration', 'administrations', 'administrative', 'administrators', 'admit', 'admitted', 'adnan', 'adopt', 'adopted', 'advance', 'advances', 'advice', 'adviser', 'advocacy', 'advocates', 'advocating', 'adwan', 'aerial', 'affairs'

## TF-IDF BM25 Okapi part

In [ ]:
import numpy as np

# def create_tf_matrix(corpus, vocab, threshold=5):
#   n_docs = len(corpus)
#   n_terms = len(vocab)

#   tf_matrix = np.zeros((n_docs, n_terms)).astype(np.int32)

#   for doc_idx, doc in enumerate(corpus):
#     words = doc.split()


#     for v_idx, v in enumerate(vocab):
#       # if the count is lower than threashold don't put it in
#       c = words.count(v)
#       if c >= threshold:
#         tf_matrix[doc_idx, v_idx] = words.count(v)

#   return tf_matrix

def create_tf_matrix(corpus, vocab, threshold=5):
  n_docs = len(corpus)
  n_terms = len(vocab)

  tf_matrix = np.zeros((n_docs, n_terms)).astype(np.int32)

  for doc_idx, doc in enumerate(corpus):
    words = doc.split()


    for v_idx, v in enumerate(vocab):
      # if the count is lower than threashold don't put it in
      c = words.count(v)
      # Instead of threshold, count all occurrences
      # if c >= threshold:
      tf_matrix[doc_idx, v_idx] = c # words.count(v)

  return tf_matrix


In [ ]:
# create the tf matrix

tf_matrix = create_tf_matrix(processed_corpus, vocab, threshold=1)

In [ ]:
# print the matrix as a df with the name of the document and the word from the vocab
documents_names = df_aj['id'].tolist()
df_tf_matrix = pd.DataFrame(tf_matrix, index=documents_names, columns=vocab)

In [ ]:
print(df_tf_matrix.head())

      aa  abadi  abbas  abc  abdallah  abdel  abductors  abide  ability  able  \
aj_1   0      0      0    0         0      0          0      0        0     0   
aj_2   0      0      0    0         0      0          0      0        0     0   
aj_3   0      0      0    0         0      0          0      0        0     0   
aj_4   0      0      0    0         0      0          0      0        0     0   
aj_5   0      0      0    0         0      0          0      0        0     0   

      ...  zakaria  zawayda  zecharia  zin  zionism  zionist  zionists  zip  \
aj_1  ...        0        0         0    0        0        0         0    0   
aj_2  ...        0        0         0    0        0        0         0    0   
aj_3  ...        0        0         0    0        0        0         0    0   
aj_4  ...        0        0         0    0        0        0         0    0   
aj_5  ...        0        0         0    0        0        0         0    0   

      zone  zones  
aj_1     0      0 

In [ ]:
# create a sparsity matrix from th tf_matrix since most of the data is 0

from scipy.sparse import csr_matrix

sparsity_matrix = csr_matrix(df_tf_matrix.values)

In [ ]:
print(sparsity_matrix.shape)
print(sparsity_matrix.size)

(599, 4393)
17720


In [ ]:
# perform the tf-idf bm25 okapi algorithm on the data

def calculate_df(tf_matrix, vocab):
    df = {}
    for term_idx in range(tf_matrix.shape[1]):
        # Check if term_idx is within vocab bounds
        if term_idx < len(vocab):
            term = vocab[term_idx]  # Get the term from the vocab
            # Get the document frequency
            # Changed from np.count_nonzero to sum of boolean array
            df[term] = np.sum(tf_matrix[:, term_idx] > 0)
    return df

df = calculate_df(tf_matrix, vocab)

In [ ]:
def calculate_avg_doc_len(corpus):
    total_len = sum(len(doc.split()) for doc in corpus)
    return total_len / len(corpus)

L_avg = calculate_avg_doc_len(processed_corpus)

In [ ]:
def tfidf_bm25_okapi(tf_matrix, df, L_avg, processed_corpus, vocab, k=1.2, b=0.75):
    M = tf_matrix.shape[0]  # Total number of documents

    tfidf_matrix = np.zeros_like(tf_matrix, dtype=np.float32)

    for doc_idx in range(tf_matrix.shape[0]):
        doc_len = len(processed_corpus[doc_idx].split())
        for term_idx in range(tf_matrix.shape[1]):
            term = vocab[term_idx]
            c_wd = tf_matrix[doc_idx, term_idx]  # Term frequency in the document

            if c_wd > 0:  # Only calculate if the term is present
                idf = np.log((M + 1) / df[term])

                numerator = (k + 1) * c_wd
                denominator = c_wd + k * (1 - b + b * (doc_len / L_avg))

                tfidf_matrix[doc_idx, term_idx] = (numerator / denominator) * idf

    return tfidf_matrix

tf_idf_matrix = tfidf_bm25_okapi(tf_matrix, df, L_avg, processed_corpus, vocab)

In [ ]:
df_tfidf_matrix = pd.DataFrame(tf_idf_matrix, index=documents_names, columns=vocab)

In [ ]:
# save the df_tfidf_matrix into a file called tfidf_aj.csv

df_tfidf_matrix.to_csv("tfidf_aj.csv")

In [ ]:
# convert to spasiti matrix

sparsity_matrix_tfidf = csr_matrix(df_tfidf_matrix.values)

In [ ]:
print(sparsity_matrix_tfidf.shape)
print(sparsity_matrix_tfidf.size)

(599, 4393)
17720


In [ ]:
def information_gain(tfidf_matrix):
    # calculate the entropy of the tfidf matrix
    entropy = -tfidf_matrix * np.log2(tfidf_matrix)
    entropy = np.sum(entropy, axis=1)
    # calculate the entropy of the tfidf matrix
    entropy = -tfidf_matrix * np.log2(tfidf_matrix)
    entropy = np.sum(entropy, axis=1)
    # calculate the information gain
    ig = np.sum(entropy) - entropy
    return ig


def gain_ratio(tfidf_matrix):
    # calculate the entropy of the tfidf matrix
    entropy = -tfidf_matrix * np.log2(tfidf_matrix)
    entropy = np.sum(entropy, axis=1)
    # calculate the information gain
    ig = np.sum(entropy) - entropy
    # calculate the gain ratio
    gr = ig / entropy
    return gr

In [ ]:
# print the top 20 terms using IG and GR

ig_values = information_gain(tf_idf_matrix)
gr_values = gain_ratio(tf_idf_matrix)

# Sort terms by IG and RG in descending order
sorted_ig_indices = np.argsort(ig_values)[::-1]  # [::-1] reverses the order (descending)
sorted_gr_indices = np.argsort(gr_values)[::-1]

# Get the top 20 terms for IG and RG
top_20_ig_terms = [vocab[i] for i in sorted_ig_indices[:20]]
top_20_gr_terms = [vocab[i] for i in sorted_gr_indices[:20]]

print("Top 20 terms by Information Gain:", top_20_ig_terms)
print("Top 20 terms by Gain Ratio:", top_20_gr_terms)


Top 20 terms by Information Gain: ['cape', 'answer', 'annual', 'announces', 'announcement', 'announced', 'announce', 'anniversary', 'annihilation', 'ankara', 'angling', 'angered', 'anger', 'angeles', 'anecdote', 'andromeda', 'andrew', 'another', 'answered', 'app']
Top 20 terms by Gain Ratio: ['cape', 'answer', 'annual', 'announces', 'announcement', 'announced', 'announce', 'anniversary', 'annihilation', 'ankara', 'angling', 'angered', 'anger', 'angeles', 'anecdote', 'andromeda', 'andrew', 'another', 'answered', 'app']


## Saving the data

In [ ]:
# save the spasity matrix

from scipy.sparse import save_npz

save_npz("sparsity_matrix_tfidf.npz", sparsity_matrix_tfidf)

In [ ]:
# save the vocab

with open("vocab.txt", "w") as f:
    for word in vocab:
        f.write(word + "\n")

In [ ]:
# open the spasity matrix for demonstration purposes

from scipy.sparse import load_npz

sparsity_matrix_tfidf_save = load_npz("sparsity_matrix_tfidf.npz")

## Loading back the data

In [ ]:
# load the vocab

with open("vocab.txt", "r") as f:
    vocab_save = f.read().splitlines()

In [ ]:
# go back to the original matrix
from scipy.sparse import csr_matrix

df_sparsity_matrix_tfidf = pd.DataFrame.sparse.from_spmatrix(sparsity_matrix_tfidf_save, columns=vocab_save)


In [ ]:
print(df_sparsity_matrix_tfidf.head())

   aa  abadi  abbas  abc  abdallah  abdel  abductors  abide  ability  able  \
0   0      0      0    0         0      0          0      0        0     0   
1   0      0      0    0         0      0          0      0        0     0   
2   0      0      0    0         0      0          0      0        0     0   
3   0      0      0    0         0      0          0      0        0     0   
4   0      0      0    0         0      0          0      0        0     0   

   ...  zakaria  zawayda  zecharia  zin  zionism  zionist  zionists  zip  \
0  ...        0        0         0    0        0        0         0    0   
1  ...        0        0         0    0        0        0         0    0   
2  ...        0        0         0    0        0        0         0    0   
3  ...        0        0         0    0        0        0         0    0   
4  ...        0        0         0    0        0        0         0    0   

   zone  zones  
0     0      0  
1     0      0  
2     0      0  
3     

In [ ]:
import requests
import pandas as pd
import os

def doanload_file(link, file_name):
  response = requests.get(link)
  if response.status_code == 200: # if the response is 200 OK
    with open(file_name, "wb") as file:
      file.write(response.content)
    print(f"Sucessfully downloaded the file into {file_name}")


# links
word_links = [aj_word_link, bbc_word_link, jp_word_link, nyt_word_link]
lemma_links = [aj_lemma_link, bbc_lemma_link, jp_lemma_link, nyt_lemma_link]

# now do all the process for all the different gorups
word_files = [aj_word_file, bbc_word_file, jp_word_file, nyt_word_file]
lemma_files = [aj_lemma_file, bbc_lemma_file, jp_lemma_file, nyt_lemma_file]

for f, l in zip(word_files, word_links):
  doanload_file(l, f)

for f, l in zip(lemma_files, lemma_links):
  doanload_file(l, f)

Sucessfully downloaded the file into A_J_word.csv
Sucessfully downloaded the file into BBC_word.csv
Sucessfully downloaded the file into J_P_word.csv
Sucessfully downloaded the file into NYT_word.csv
Sucessfully downloaded the file into A_J_lemma.csv
Sucessfully downloaded the file into BBC_lemma.csv
Sucessfully downloaded the file into J_P_lemma.csv
Sucessfully downloaded the file into NYT_lemma.csv


## Process Pipeline

1. read `csv`.
2. create corpuse `corpus = df['document'].tolist()`.
3. process the corpus. (remove stop-words etc).
4. create the vocab.
5. create the TF matrix.
6. create the DF matrix.
7. calculate the $L_{avg}$.
8. Create & calculate the $\text{TF-IDF}$ matrix with BM25 and Okapi.
9. Save the matrix with the `.pkz` format.
10. Perform the $\text{Information Gain}$ & $\text{Gaint Ratio}$.

In [ ]:
# 1. Read CSV
dfs_word = {}
dfs_lemma = {}

for f in word_files:
  df = pd.read_csv(f)
  dfs_word[f] = df

for f in lemma_files:
  df = pd.read_csv(f)
  dfs_lemma[f] = df

In [ ]:
# 2. Create corpuses + 3. process the corpus + 4. create the vocab
corpuses = {}

vocabs = {}

for f, df in dfs_word.items():
  print(f"Building corpus for file:{f}")
  corpus = df['document'].tolist()
  processed_corpus = process_corpus(corpus)
  corpuses[f] = processed_corpus
  # processing the vocab
  vocab = get_vocab(processed_corpus)
  vocabs[f] = vocab

for f, df in dfs_lemma.items():
  print(f"Building corpus for file:{f}")
  corpus = df['document'].tolist()
  processed_corpus = process_corpus(corpus)
  corpuses[f] = processed_corpus
  # processing the vocab
  vocab = get_vocab(processed_corpus)
  vocabs[f] = vocab

Building corpus for file:A_J_word.csv
Building corpus for file:BBC_word.csv
Building corpus for file:J_P_word.csv
Building corpus for file:NYT_word.csv
Building corpus for file:A_J_lemma.csv
Building corpus for file:BBC_lemma.csv
Building corpus for file:J_P_lemma.csv
Building corpus for file:NYT_lemma.csv


In [ ]:
# 5. create the TF matrix + 6. create the DF matrix + 7. L_avg
# + 8. TF-IDF BM25 Okapi

tf_idf_matrices = {}

for f in corpuses.keys():
  print(f"Building TF-IDF for file: {f}")
  tf_matrix = create_tf_matrix(corpuses[f], vocabs[f])
  df_matrix = calculate_df(tf_matrix, vocabs[f])
  L_avg = calculate_avg_doc_len(corpuses[f])
  tf_idf_matrix =  tfidf_bm25_okapi(tf_matrix, df_matrix,
                                    L_avg, corpuses[f], vocabs[f])

  tf_idf_matrices[f] = tf_idf_matrix



Building TF-IDF for file: A_J_word.csv
Building TF-IDF for file: BBC_word.csv
Building TF-IDF for file: J_P_word.csv
Building TF-IDF for file: NYT_word.csv
Building TF-IDF for file: A_J_lemma.csv
Building TF-IDF for file: BBC_lemma.csv
Building TF-IDF for file: J_P_lemma.csv
Building TF-IDF for file: NYT_lemma.csv


In [ ]:
# perform the Gain Ration + Information Gain

for f, mat in tf_idf_matrices.items():
  print(f"Building IG and GR for file:{f}")
  ig_values = information_gain(mat)
  gr_values = gain_ratio(mat)

  # Sort terms by IG and RG in descending order
  sorted_ig_indices = np.argsort(ig_values)[::-1]  # [::-1] reverses the order (descending)
  sorted_gr_indices = np.argsort(gr_values)[::-1]

  # Get the top 20 terms for IG and RG
  top_20_ig_terms = [vocab[i] for i in sorted_ig_indices[:20]]
  top_20_gr_terms = [vocab[i] for i in sorted_gr_indices[:20]]

  print("Top 20 terms by Information Gain:\n", top_20_ig_terms)
  print("Top 20 terms by Gain Ratio:\n", top_20_gr_terms)

Building IG and GR for file:A_J_word.csv
Top 20 terms by Information Gain:
 ['barricade', 'algerias', 'alfitr', 'alexius', 'alexandria', 'alert', 'aleaziz', 'ale', 'aldaqran', 'albaressaid', 'albany', 'albalah', 'albag', 'alawda', 'alassad', 'alarmed', 'alarm', 'algeria', 'ali', 'allow']
Top 20 terms by Gain Ratio:
 ['barricade', 'algerias', 'alfitr', 'alexius', 'alexandria', 'alert', 'aleaziz', 'ale', 'aldaqran', 'albaressaid', 'albany', 'albalah', 'albag', 'alawda', 'alassad', 'alarmed', 'alarm', 'algeria', 'ali', 'allow']
Building IG and GR for file:BBC_word.csv
Top 20 terms by Information Gain:
 ['badfaith', 'ajoint', 'alan', 'alam', 'alaliin', 'alali', 'ala', 'al', 'airstrikein', 'algerias', 'airstrike', 'airportstyle', 'airlift', 'airdrop', 'aircraft', 'airborne', 'alaqsa', 'alarge', 'alarm', 'alarmed']
Top 20 terms by Gain Ratio:
 ['badfaith', 'ajoint', 'alan', 'alam', 'alaliin', 'alali', 'ala', 'al', 'airstrikein', 'algerias', 'airstrike', 'airportstyle', 'airlift', 'airdrop', 

In [ ]:
# 9. save the data in the .pkz format

from scipy.sparse import save_npz, csr_matrix # Import csr_matrix

def save_matrix(file_name, tf_idf_matrix, vocab):
  # Convert tf_idf_matrix to a sparse matrix before saving
  tf_idf_matrix_sparse = csr_matrix(tf_idf_matrix)
  save_npz(f"{file_name}_spasity.npz", tf_idf_matrix_sparse)

  # save the vocab as a txt file

  with open(f"{file_name}_vocab.txt", "w") as f:
    for word in vocab:
        f.write(word + "\n")

  print(f"save file: {file_name}")

In [ ]:
# save all the matrices

file_names = ['aj_word', 'bbc_word', 'jp_word', 'nyt_word',
              'aj_lemma', 'bbc_lemma', 'jp_lemma', 'nyt_lemma']

for i, pair in enumerate(tf_idf_matrices.items()):
  f, matrix = pair
  file_name = file_names[i]
  print(f"Saving {file_name} tf-idf_matrix")
  save_matrix(file_name, matrix, vocabs[f])


Saving aj_word tf-idf_matrix
save file: aj_word
Saving bbc_word tf-idf_matrix
save file: bbc_word
Saving jp_word tf-idf_matrix
save file: jp_word
Saving nyt_word tf-idf_matrix
save file: nyt_word
Saving aj_lemma tf-idf_matrix
save file: aj_lemma
Saving bbc_lemma tf-idf_matrix
save file: bbc_lemma
Saving jp_lemma tf-idf_matrix
save file: jp_lemma
Saving nyt_lemma tf-idf_matrix
save file: nyt_lemma


In [ ]:
# load back the tf-idf matrix into a pandas df
def load_matrix(file_name):
  sparsity_matrix_tfidf_save = load_npz(f"{file_name}_spasity.npz")
  with open(f"{file_name}_vocab.txt", "r") as f:
    vocab_save = f.read().splitlines()

  # cerate a df
  df_sparsity_matrix_tfidf = pd.DataFrame.sparse.from_spmatrix(sparsity_matrix_tfidf_save, columns=vocab_save)

  return df_sparsity_matrix_tfidf



In [ ]:
# test the load function on a file

df_test = load_matrix("aj_word")

df_test.head()

,aa,abadi,abbas,abc,abdallah,abdel,abductors,abide,ability,able,...,zakaria,zawayda,zecharia,zin,zionism,zionist,zionists,zip,zone,zones
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
